## Naive RAG

### Load API Keys

In [1]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

### Setup Langsmith Tracking and API Key

In [2]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"]="NAIVE_RAG"

## Load LLM model from OpenAI

In [3]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4.1-nano",
                    api_key=OPENAI_API_KEY,
                    temperature=0.3,
                    max_tokens=512 )

### Test LLM 

In [4]:
test_llm_response=llm.invoke("What is Large Language Models")
test_llm_response

AIMessage(content="Large Language Models (LLMs) are advanced artificial intelligence systems designed to understand, generate, and manipulate human language. They are built using deep learning techniques, particularly neural networks with many layers, and are trained on vast amounts of text data from books, websites, and other sources. This extensive training enables LLMs to grasp complex language patterns, context, and nuances, allowing them to perform a variety of tasks such as answering questions, translating languages, summarizing texts, generating creative writing, and more. Examples of LLMs include OpenAI's GPT series, Google's BERT, and others that have significantly advanced natural language processing capabilities.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 12, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_predictio

## Load Text embedding model from OpenAI

In [5]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

### Test Embedding model

In [6]:
embedding_vector=embedding_model.embed_query("What is Large Language Models")

In [7]:
len(embedding_vector)

1536

## Load Documents

### CSV Loader

In [8]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path="sample_docs/ElectroTV_Sales_Report_2024.csv")

csv_data = loader.load()

In [9]:
print(csv_data)

[Document(metadata={'source': 'sample_docs/ElectroTV_Sales_Report_2024.csv', 'row': 0}, page_content='order_id: ORD00165\ndate: 2024-11-02\nproduct_name: ElectroTV E32 Smart\nunits_sold: 2\nunit_price_inr: 14999\ntotal_sales_inr: 29998\nsales_region: Central\nsales_channel: Online'), Document(metadata={'source': 'sample_docs/ElectroTV_Sales_Report_2024.csv', 'row': 1}, page_content='order_id: ORD00028\ndate: 2024-01-06\nproduct_name: ElectroTV E32 Smart\nunits_sold: 3\nunit_price_inr: 14999\ntotal_sales_inr: 44997\nsales_region: Central\nsales_channel: Distributor'), Document(metadata={'source': 'sample_docs/ElectroTV_Sales_Report_2024.csv', 'row': 2}, page_content='order_id: ORD00451\ndate: 2024-03-04\nproduct_name: ElectroTV E32 Smart\nunits_sold: 3\nunit_price_inr: 14999\ntotal_sales_inr: 44997\nsales_region: Central\nsales_channel: Retail Store'), Document(metadata={'source': 'sample_docs/ElectroTV_Sales_Report_2024.csv', 'row': 3}, page_content='order_id: ORD00240\ndate: 2024-08-2

### PDF Loader

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader(file_path="sample_docs/ElectroTV..pdf")

pdf_data = loader.load()

In [11]:
print(pdf_data)

[Document(metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-12-25T14:00:02+05:30', 'source': 'sample_docs/ElectroTV..pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='ElectroTV\nAbout Us:\nElectroTV is a consumer electronics brand established with the singular purpose of redefining how  \nmodern households experience television and digital entertainment. From its inception, ElectroTV  \nhas focused on designing products that balance performance, reliability, and accessibility, ensuring  \nthat advanced display technologies are not limited to premium price segments alone. The company  \noperates at the intersection of engineering precision and user-centered design, allowing it to create  \ntelevisions that are not only technologically capable but also intuitive and dependable in everyday  \nuse.\nThe foundation of ElectroTV is built on a deep understanding of how televisions are used in real-\nworld  environments.  Unlike  brands  t

### Merge CSV and PDF data

In [12]:
documents = csv_data + pdf_data

In [13]:
documents[0]

Document(metadata={'source': 'sample_docs/ElectroTV_Sales_Report_2024.csv', 'row': 0}, page_content='order_id: ORD00165\ndate: 2024-11-02\nproduct_name: ElectroTV E32 Smart\nunits_sold: 2\nunit_price_inr: 14999\ntotal_sales_inr: 29998\nsales_region: Central\nsales_channel: Online')

In [14]:
documents[-1]

Document(metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-12-25T14:00:02+05:30', 'source': 'sample_docs/ElectroTV..pdf', 'total_pages': 11, 'page': 10, 'page_label': '11'}, page_content='Andheri East,\nMumbai – 400069, Maharashtra\n Phone: +91-22-4455-9900📞\n Email: west.sales@electrotv.com📧\nEast Region Office\nElectroTV Regional Office – East\nInfinity IT Park, Block B,\nSalt Lake Sector V ,\nKolkata – 700091, West Bengal\n Phone: +91-33-4098-1122📞\n Email: east.support@electrotv.com📧\nCustomer Care & Service Support\nFor product installation, troubleshooting, warranty information, and service requests, customers \nmay contact our centralized support team.\n Toll-Free: 1800-555-ETV1 (1800-555-3881)📞\n Email: support@electrotv.com📧\n Website: www.electrotv.com 🌐 (fictional)\n11')

## Document Splitting

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(documents)

In [16]:
len(chunks)

542

In [17]:
chunks[0].page_content

'order_id: ORD00165\ndate: 2024-11-02\nproduct_name: ElectroTV E32 Smart\nunits_sold: 2\nunit_price_inr: 14999\ntotal_sales_inr: 29998\nsales_region: Central\nsales_channel: Online'

In [18]:
chunks[-1].page_content

'may contact our centralized support team.\n Toll-Free: 1800-555-ETV1 (1800-555-3881)📞\n Email: support@electrotv.com📧\n Website: www.electrotv.com 🌐 (fictional)\n11'

### Add ids to chunks

In [19]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(chunks))]

In [20]:
len(uuids)

542

In [21]:
uuids[:5]

['20d6511f-ba8f-4ff6-8803-1d91c3b63643',
 'b2b6898a-0dce-442d-bd9b-9a821038bad1',
 'a85fad24-878d-4467-a0b5-a482831c6992',
 '1ddfd788-7f45-43ec-ae29-1529bc6dd730',
 '3aa19162-9b32-4ca8-92c5-22d91015072a']

## Vector Store: Chroma db

### Initialization

In [22]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="ElectroTV",
    embedding_function=embedding_model,
    persist_directory="/home/abhishek/ad-workspace/chroma_db/ElectroTV",
)

### Add Chunks & ids

In [23]:
vector_store.add_documents(documents=chunks, ids=uuids)

['20d6511f-ba8f-4ff6-8803-1d91c3b63643',
 'b2b6898a-0dce-442d-bd9b-9a821038bad1',
 'a85fad24-878d-4467-a0b5-a482831c6992',
 '1ddfd788-7f45-43ec-ae29-1529bc6dd730',
 '3aa19162-9b32-4ca8-92c5-22d91015072a',
 '617f6dee-1016-4197-9f90-2eb0af67d28e',
 '96fd53b1-95c4-4e5d-87d7-123808aa4162',
 'd30f3908-4896-4ac5-8c98-95d115935cf7',
 '8524625c-507c-4fd6-acb5-e7b26fb598ff',
 '8e3421c5-fee2-4a18-aafd-8920ca7e505c',
 '765d46da-0106-4bb1-a551-79a11091529d',
 '85b26f4d-1882-4776-b4dc-0d1723786b48',
 'c738660a-ca29-426a-b8e7-cd9f6feb4d5b',
 'bfcf410d-4ca2-44d0-96eb-ee9334116385',
 '98d6bfd8-9310-4385-9ec8-3dccb5180502',
 'be809d9e-6ab3-4d41-9d84-9c192f9d7af3',
 '58941479-fdd9-403f-a7a2-c550c62bd226',
 '6eabdac0-0607-4e61-b884-bce292fa94e6',
 '56e51bd2-5482-4f0e-bc6f-fccf75fbbfa6',
 '7944ccf1-c297-4781-83ff-0bf7749b3922',
 '4cc2daae-fb2b-4e67-a936-6c152eede91c',
 'b7c5a527-fdd0-4f19-8458-5462d7b0b99b',
 '6e4c68b6-f755-467e-a1ea-8b34c000eb53',
 'acfd8d2a-9cca-4fdb-a365-efbdc457f9ab',
 '889b5bd1-9c07-

### Create Retriever

In [24]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

## Test Similarily Search [OPTIONAL]

### Test Query

In [25]:
test_query = "Where is the Head office of ElectroTV"

### Similarity Search

In [26]:
similar_docs = vector_store.similarity_search(test_query,k=3)

In [27]:
for i in range(len(similar_docs)):
    print("=====================\n")
    print("Similar doc : " + str(i))
    print("=====================\n")
    print(similar_docs[i].page_content)


Similar doc : 0

Contact Us
ElectroTV welcomes inquiries from customers, partners, and business stakeholders regarding our 
products, services, and support offerings. Our corporate and regional offices are structured to ensure 
prompt assistance, transparent communication, and efficient resolution of queries. Whether you are 
seeking product information, sales support, or after-sales service, our teams are available through 
the contact details provided below.
Head Office (Corporate Headquarters)

Similar doc : 1

the contact details provided below.
Head Office (Corporate Headquarters)
ElectroTV Electronics Pvt. Ltd.
ElectroTV Tower, Plot No. 42,
Tech Park Avenue, Sector 18,
Gurugram – 122015, Haryana, India
 Phone: +91-11-4567-8900📞
 Email: corporate@electrotv.com📧
Regional Branch Offices
North Region Office
ElectroTV Regional Office – North
2nd Floor, Orion Business Center,
Noida Sector 62, Uttar Pradesh – 201309
 Phone: +91-120-678-2345📞
 Email: north.sales@electrotv.com📧
South Reg

## Naive RAG Pipeline

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Answer the question using ONLY the context below.
If you don't know the answer based on the context, say you don't know.

Context:
{context}

Question:
{question}
""")

naive_rag_chain = (
    {
        "context": retriever,
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)


## Question and Answer

In [29]:
question = "Where is the head office of ElectroTV"
print(naive_rag_chain.invoke(question))

The head office of ElectroTV is located at ElectroTV Tower, Plot No. 42, Tech Park Avenue, Sector 18, Gurugram – 122015, Haryana, India.


In [30]:
question = "How many regional officies does ElectroTV has and give me their contact numbers"
print(naive_rag_chain.invoke(question))

ElectroTV has three regional offices. Their contact numbers are:

- North Region Office: +91-120-678-2345
- South Region Office: +91-80-5123-7788
- West Region Office: (Contact number not fully provided in the context)


In [31]:
question = "How many Televion models are launched by ElectroTV so far. List down all of them"
print(naive_rag_chain.invoke(question))

Based on the provided context, two television models are mentioned as launched by ElectroTV:
1. ElectroTV E43 Smart - Family-oriented TV
2. ElectroTV E50 Pro - Entry-level 4K TV


In [32]:
question = "Which ElectroTV products are QLED televisions and what are their listed prices?"
print(naive_rag_chain.invoke(question))

The ElectroTV products that are QLED televisions are:

- ElectroTV Q55 Ultra, priced at 75,000₹
- ElectroTV Q55 Ultra, priced at 54,999₹
- ElectroTV Q65 Ultra, priced at 69,999₹


In [33]:
question = "What is the phone number and email address of the ElectroTV Head Office?"
print(naive_rag_chain.invoke(question))

The phone number of the ElectroTV Head Office is +91-11-4567-8900, and the email address is corporate@electrotv.com.


In [34]:
question = "Which ElectroTV product is identified as the flagship model, and what feature justifies this positioning?"
print(naive_rag_chain.invoke(question))

I don't know.


In [35]:
question = "Which ElectroTV models priced below ₹50,000 are available, and how many total units of these models were sold in 2024?"
print(naive_rag_chain.invoke(question))

The ElectroTV models priced below ₹50,000 that are available are the ElectroTV E40 Smart and the ElectroTV E50 Pro. 

Total units sold in 2024:
- ElectroTV E40 Smart: 15 units
- ElectroTV E50 Pro: 2 units + 9 units = 11 units

Total units sold of these models in 2024: 15 + 11 = 26 units.


In [36]:
question = "Which ElectroTV product generated the highest total revenue across all sales records?"
print(naive_rag_chain.invoke(question))

The ElectroTV product that generated the highest total revenue across all sales records is the ElectroTV E55 Pro+.


In [37]:
question = "Which sales channel contributed the most revenue for premium-priced ElectroTV televisions (price above ₹60,000)?"
print(naive_rag_chain.invoke(question))

Based on the provided data, the premium-priced ElectroTV televisions (price above ₹60,000) are:

- ElectroTV Q65 Ultra with a unit price of ₹69,999 and total sales of ₹979,986 (West region, Online channel).

Since only one such product is listed, the sales channel contributing the most revenue for premium-priced ElectroTV televisions is the Online channel.


In [38]:
question = "Which ElectroTV products are marketed for home cinema use, and did they collectively generate more revenue than QLED models?"
print(naive_rag_chain.invoke(question))

I don't know.


In [39]:
question = "Which ElectroTV regional office should a customer in Bengaluru contact, and what are the contact details?"
print(naive_rag_chain.invoke(question))

A customer in Bengaluru should contact the ElectroTV South Region Office. The contact details are:

ElectroTV Regional Office – South  
Sigma Tech Plaza, 5th Floor,  
Whitefield Main Road,  
Bengaluru – 560066, Karnataka  
Phone: +91-80-5123-7788📞  
Email: south.support@electrotv.com📧


In [40]:
question = "For E32 Smart model, Give me the following details: Price, Features "
print(naive_rag_chain.invoke(question))

Price: 14,999 INR  
Features: Android TV, HD Ready Display, 20W Sound Output


In [41]:
question = "How many units of ElectroTV E32 Smart are sold in Northern region in the year 2024 "
print(naive_rag_chain.invoke(question))

I don't know.
